In [95]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [96]:
# Reading the CSV file and splitting it into feature and label matrices

dataset = pd.read_csv('data_banknote_authentication.csv')
dataset = dataset.to_numpy()

X = dataset[:,0:-1]
Y = dataset[:,-1]

# Splitting the dataset into training and testing data in the ratio 7:3

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
y_train = np.reshape(y_train, (X_train.shape[0],1))
X_train = np.append(X_train, y_train, axis=1)

In [97]:
# Seperating the features into respective classes (0 and 1)

class1 = [] # should have dataset only belonging to inauthentic banknotes
class2 = [] # should have dataset only belonging to authentic banknotes

for i in range(X_train.shape[0]):
  if X_train[i][-1] == 0:
      class1.append(X_train[i])
  else:
      class2.append(X_train[i])

# Function to calculate Gini Index

In [98]:
def gini_index(regions, classes):
    
    region1 = regions[0]
    region2 = regions[1]
    n1 = len(region1)
    n2 = len(region2)

    Q1 = 0
    for cls in classes:
        count = 0
        for pt in region1:
            if (pt[-1] == cls):
                count += 1
        prop = 0
        if (n1 != 0):
            prop = count/n1
        Q1 += prop*(1-prop)

    Q2 = 0
    for cls in classes:
        count = 0
        for pt in region2:
            if (pt[-1] == cls):
                count += 1
        prop = 0
        if (n2 != 0):
            prop = count/n2
        Q2 += prop*(1-prop)

    Gini = (n1*Q1 + n2*Q2)/(n1+n2)

    return Gini

# Function to Create Split

In [99]:
# Split a dataset based on an input feature and a input feature value
def split_at_root_node(feature_index, value, dataset): 

    left = []
    right = []

    for data in dataset:
        if (data[feature_index] <= value):
            left.append(data)
        else:
            right.append(data)

    return left, right

# Function to get best split in a region

In [110]:
# Select the best split point for a given dataset
def get_best_split(dataset):

	features = np.linspace(0,len(dataset[0])-2,num=len(dataset[0])-1,dtype=int) 
	best_index = -1
	best_value = float('inf')
	best_gini_score = float('inf')
	best_regions = []

	for index in features:
		values = [row[index] for row in dataset]
		set_values = set(values)
		unique_values = list(set_values)
		unique_values.sort()
		for i in range(len(unique_values)-1):
			val = (unique_values[i]+unique_values[i+1])/2
			L, R = split_at_root_node(index, val, dataset)
			gini = gini_index([L, R], [0, 1])
			#print('x', index, '<', val, 'gini: ', gini)
			if (gini < best_gini_score):
				best_gini_score = gini
				best_index = index
				best_value = val
				best_regions = [L, R]

	return {'index':best_index, 'value':best_value, 'gini': best_gini_score, 'regions':best_regions}

# Function to assign value to a leaf node

In [120]:
# Create a leaf value
def leaf_output(region):
  outcomes = [row[-1] for row in region]
  return max(set(outcomes), key=outcomes.count)

# Function to create internal child splits with min_size and max_depth constraints

In [125]:
# Creates internal child splits for a node or make a leaf node
def split(internal_node, max_depth, min_size, depth): 
  left = internal_node['regions'][0]
  right = internal_node['regions'][1]
  if not left or not right:
    internal_node['left'] = internal_node['right'] = leaf_output(left + right)
  elif (depth == max_depth) or (min_size >= (len(left) + len(right))):
    internal_node['left'] = leaf_output(left)
    internal_node['right'] = leaf_output(right)
  else:
    left_child = get_best_split(left)
    right_child = get_best_split(right)
    if left_child['index'] == -1:
      internal_node['left'] = left[0][-1]
    else:
      internal_node['left'] = left_child
      split(internal_node['left'], max_depth, min_size, depth+1)
    if right_child['index'] == -1:
      internal_node['right'] = right[0][-1]
    else:
      internal_node['right'] = right_child
      split(internal_node['right'], max_depth, min_size, depth+1)

# Function to build the decision tree

In [126]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
  root = get_best_split(train)
  split(root, max_depth, min_size, 0)
  return root

# Function to print the built decision tree

In [127]:
# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

In [128]:
tree = build_tree(X_train, 5, 10)
print_tree(tree)

[X1 < 0.321]
 [X2 < 7.565]
  [X1 < -1.741]
   [X2 < 1.772]
    [X1 < -5.208]
     [1.0]
     [X1 < -5.095]
      [1.0]
      [1.0]
    [X2 < 1.775]
     [0.0]
     [X1 < -6.568]
      [1.0]
      [1.0]
   [X3 < 6.824]
    [X2 < 3.925]
     [X3 < 6.216]
      [1.0]
      [1.0]
     [X3 < -3.602]
      [1.0]
      [0.0]
    [X2 < -6.808]
     [X1 < -1.721]
      [1.0]
      [1.0]
     [X1 < -1.715]
      [0.0]
      [0.0]
  [X1 < -3.954]
   [X1 < -7.039]
    [1.0]
    [X1 < -6.998]
     [1.0]
     [X1 < -6.742]
      [1.0]
      [1.0]
   [X1 < -2.728]
    [0.0]
    [X1 < -2.681]
     [0.0]
     [X1 < -2.572]
      [0.0]
      [0.0]
 [X3 < -4.394]
  [X1 < 3.304]
   [X1 < 0.403]
    [1.0]
    [X1 < 0.545]
     [1.0]
     [X1 < 0.627]
      [1.0]
      [1.0]
   [X1 < 4.312]
    [0.0]
    [0.0]
  [X1 < 1.450]
   [X3 < -2.243]
    [X1 < 0.543]
     [1.0]
     [X1 < 0.580]
      [1.0]
      [1.0]
    [X4 < 0.082]
     [X1 < 0.420]
      [0.0]
      [0.0]
     [X3 < 0.324]
      [1.0]
      [0.

# Function to predict using the decision tree

In [129]:
# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [131]:
misclassified = 0
prediction = []

for i in range(X_train.shape[0]):
    output = predict(tree, X_train[i])
    prediction.append(output)
    if (prediction[i] != y_train[i]):
        misclassified += 1

print("Accuracy on training set = ", 100*(1 - misclassified/y_train.shape[0]), "%")


misclassified = 0
prediction = []

for i in range(X_test.shape[0]):
    output = predict(tree, X_test[i])
    prediction.append(output)
    if (prediction[i] != y_test[i]):
        misclassified += 1

print("Accuracy on testing set = ", 100*(1 - misclassified/y_test.shape[0]), "%")

Accuracy on training set =  99.68717413972888 %
Accuracy on testing set =  97.57281553398059 %
